<h1 style='color:red; text-align:center;'>Stock prediction using machine learning and python</h1>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import keras

In [ ]:
path='./Data/Google_Stock_Price_Train.csv'
stock_data = pd.read_csv(path,index_col="Date",parse_dates=True)
stock_data.head(10)

In [ ]:
stock_data.tail()

<h2>Data Pre-processing</h2>

In [ ]:
#shape of data
stock_data.shape

In [ ]:
stock_data.head()

In [ ]:
stock_data.isnull().sum()
stock_data.isna().any()
#no nan or missing data hence data is cleaned

In [ ]:
stock_data.info()

In [ ]:
stock_data['Close'] = stock_data['Close'].str.replace(',','').astype(float)

In [ ]:
stock_data['Volume'] = stock_data['Volume'].str.replace(',','').astype(float)

In [ ]:
stock_data['Open'].plot(figsize=(16,6))
plt.title('Growth of stocks from 2012 to 2017')
plt.show()

In [ ]:
#7 days rolling mean
stock_data.rolling(7).mean().head(20)

In [ ]:
stock_data['Open'].plot()
stock_data.rolling(window=30).mean()['Close'].plot()
plt.title("Comparing rolling mean and growth")
plt.show()

In [ ]:
stock_data['Close:30 Day mean']=stock_data['Close'].rolling(window=30).mean()
stock_data[['Close','Close:30 Day mean']].plot(figsize=(16,6))
plt.show()

In [ ]:
stock_data['Close'].expanding(min_periods=1).mean().plot()
plt.show()

In [ ]:
training_data= stock_data['Open']
training_data=pd.DataFrame(training_data)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sc=MinMaxScaler(feature_range=(0,1))
training_data_scaled=sc.fit_transform(training_data)

In [ ]:
#60 times and one output
X_train = []
Y_train = []
for i in range(60,1235):
    X_train.append(training_data_scaled[i-60:i,0])
    Y_train.append(training_data_scaled[i,0])
X_train,Y_train=np.array(X_train),np.array(Y_train)
#reshaping
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
# X_train.shape

<h2>Building RNN </h2>

In [ ]:
#Building RNN
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
regression = Sequential()

In [ ]:
#Adding the first layer and some Dropout regularisation
regression.add(LSTM(units=50,return_sequences=True ,input_shape=(X_train.shape[1],1)))
regression.add(Dropout(0.2))

#Adding the second layer and some Dropout regularisation
regression.add(LSTM(units=50,return_sequences=True ))
regression.add(Dropout(0.2))

#Adding the third layer and some Dropout regularisation
regression.add(LSTM(units=50,return_sequences=True ))
regression.add(Dropout(0.2))

#Adding the third layer and some Dropout regularisation
regression.add(LSTM(units=50))
regression.add(Dropout(0.2))

#Adding output layer
regression.add(Dense(units=1))

<h2>Training RNN</h2>

In [ ]:
#compiling the rnn
regression.compile(optimizer='adam',loss='mean_squared_error')

#fitting the rnn into Training set
regression.fit(X_train,Y_train,epochs=100,batch_size=32)

<h2>Testing RNN</h2>

In [ ]:
test_data = pd.read_csv('Google_Stock_Price_Test.csv',index_col="Date",parse_dates=True)

In [ ]:
real_stock_price = test_data.iloc[:,1:2].values
real_stock_price

In [ ]:
test_data.head()

In [ ]:
test_data.info()

In [ ]:
test_data['Volume'] = test_data['Volume'].str.replace(',','').astype(float)

In [ ]:
test_data.info()

In [ ]:
test_set = test_data['Open']
test_set = pd.DataFrame(test_set)

In [ ]:
#Getting the predicted stock price of 2018
dataset_total = pd.concat((stock_data['Open'],test_data['Open']),axis=0)
inputs = dataset_total[len(dataset_total)-len(test_data)-60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
x_test=[]
for i in range(60,80):
    x_test.append(inputs[i-60:i,0])
x_test = np.array(x_test)
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
predicted_stock_price = regression.predict(x_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
predicted_stock_price = pd.DataFrame(predicted_stock_price)
predicted_stock_price.info()

<h2>Data visulisation</h2>

In [ ]:
plt.plot(real_stock_price,color='red',label="Real Stock Price")
plt.plot(predicted_stock_price,color='blue',label="Predicted Stock Price")
plt.title("Google stock price prediction")
plt.xlabel("Time")
plt.ylabel("stock price")
plt.legend()
plt.show()